In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import get_raw_data
import get_preprocessing_data
import analysis_method
import statsmodels.regression.linear_model as sm

from matplotlib import font_manager, rc
font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)

pv_list = ["0102", "1701", "1702", "0111", "0330", "0331", "0332", "0333", "0101", "0107", "0110", "5501"]
#["0102"(가구주와의 관계), "1701"(근로소득 유무), "1702"(총 연간근로소득), "0111"(학력졸업여부), "0330"(업종8차),
# "0331"(업종9차), "0332"(직종8차), "0333"(직종9차), "0101"(성별), "0107"(만나이), "0110"(학력학교), "5501"(혼인상태)]
hv_list = ["0150", "1406", "2102", "2134", "2136", " 2138", "2140", "2142", "2152", "2153", "2155",
           "2156", "2157", "2158", "2159", "2160", "2182", "2183", "2184", "2185", "2186", "2187",
           "2189", "2190", "2191"]
income_code_list = ["2102", "2134", "2136", " 2138", "2140", "2142", "2152", "2153", "2155", "2156",
                    "2157", "2158", "2159", "2160", "2182", "2183", "2184", "2185", "2186", "2187",
                    "2189", "2190", "2191"]

cpi_data = pd.read_excel("C:\\Users\\JeayongYu\\PycharmProjects\\human_captial\\data\\CPI.xls").loc[0, :]
cpi_data_index = pd.read_excel("C:\\Users\\JeayongYu\\PycharmProjects\\human_captial\\data\\CPI.xls", dtype="str").loc[14, :].values[1:]
cpi_data = pd.Series(cpi_data.values[1:], dtype="float")
cpi_data.name = "CPI"
cpi_data.index = cpi_data_index

industry_8 = pd.read_excel("C:\\Users\\JeayongYu\\PycharmProjects\\human_captial\\data\\국내데이터_산업분류.xlsx", sheet_name="8차")
industry_large = pd.read_excel("C:\\Users\\JeayongYu\\PycharmProjects\\human_captial\\data\\국내데이터_산업분류.xlsx", sheet_name="대분류")

kospi_return = pd.read_excel("C:\\Users\\JeayongYu\\PycharmProjects\\human_captial\\data\\kospi_return.xlsx", index_col=0)

data_num = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20", "21"]

cpi_multiple = get_preprocessing_data.get_cpi_multi(cpi_data, data_num)

data_dict = {}
for i in data_num:
    h_path = "C:\\Users\\JeayongYu\\PycharmProjects\\human_captial\\data\\h_data\\h_" + i + ".pickle"
    p_path = "C:\\Users\\JeayongYu\\PycharmProjects\\human_captial\\data\\p_data\\p_" + i + ".pickle"

    h_data = pd.read_pickle(h_path)
    p_data = pd.read_pickle(p_path)

    raw_hdata = get_raw_data.get_hv_data(h_data, hv_list, i)
    raw_pdata = get_raw_data.get_pv_data(p_data, pv_list, i)
    
    fin_p_data = get_preprocessing_data.get_fin_personal(raw_pdata, industry_8, industry_large)
    income_data = get_preprocessing_data.get_income_data(fin_p_data.index, raw_hdata)

    fin_x_value = get_preprocessing_data.get_final_xvalue(raw_hdata, fin_p_data)
    fin_y_value = get_preprocessing_data.get_final_yvalue(income_data, cpi_multiple[i])

    finally_preprocessed_data = get_preprocessing_data.get_total_data(fin_y_value, fin_x_value)

    data_dict[i] = finally_preprocessed_data

C:\Users\JeayongYu\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
data_df = pd.DataFrame()
for i in data_dict.keys():
    data_df = pd.concat([data_df, data_dict[i]])

_______________________________


### log(Y*)의 계산

In [3]:
temp_edu = data_df.groupby(["0110", "jobtype"]).get_group((1, "employ"))
temp_edu = get_preprocessing_data.get_agecut_data(temp_edu, 22)
x_mat = analysis_method.get_reg_x(temp_edu).iloc[:, :3]
fer_results = analysis_method.fe_regression_sol(temp_edu, False)

poly_x_mat = analysis_method.polynomial_matrix(temp_edu["0107"], 3) .iloc[:, 1:]
poly_x_mat.index = x_mat.index
fitting_result = analysis_method.fitting_polynomial(fer_results, 3, False)

In [4]:
fer_results[0][:3]

intercept     7.492620
marriage      0.330761
family_num    0.058637
dtype: float64

In [5]:
fitting_result[1]

intercept    -0.390035
age^1/10^0   -0.013523
age^2/10^1    0.012117
age^3/10^2   -0.001289
dtype: float64

In [6]:
fitting_paramters = pd.concat([fer_results[0][:3], fitting_result[1]], axis=1).sum(axis=1)
fitting_paramters

intercept     7.102585
marriage      0.330761
family_num    0.058637
age^1/10^0   -0.013523
age^2/10^1    0.012117
age^3/10^2   -0.001289
dtype: float64

In [7]:
real_data = pd.concat([x_mat, poly_x_mat], axis=1)
real_data

,intercept,marriage,family_num,age^1/10^0,age^2/10^1,age^3/10^2
hhid,,,,,,
21.0,1,1.0,3.0,25.0,62.5,156.25
68.0,1,1.0,4.0,56.0,313.6,1756.16
74.0,1,1.0,4.0,42.0,176.4,740.88
77.0,1,1.0,4.0,44.0,193.6,851.84
80.0,1,1.0,4.0,53.0,280.9,1488.77
...,...,...,...,...,...,...
110252.0,1,0.0,1.0,24.0,57.6,138.24
110064.0,1,1.0,4.0,43.0,184.9,795.07
110085.0,1,1.0,4.0,35.0,122.5,428.75


In [8]:
temp_edu["labor income"]

hhid
21.0        1940.968781
68.0        2329.162537
74.0        3299.646928
77.0        2911.453171
80.0        5822.906343
               ...     
110252.0    2760.000000
110064.0    5300.000000
110085.0    6140.000000
110102.0    3500.000000
110122.0    6678.000000
Name: labor income, Length: 14995, dtype: float64

In [9]:
real_data @ fitting_paramters

hhid
21.0        7.827010
68.0        8.446226
74.0        8.282097
77.0        8.320402
80.0        8.435313
              ...   
110252.0    7.356347
110064.0    8.301701
110085.0    8.126089
110102.0    7.887310
110122.0    8.173243
Length: 14995, dtype: float64

_______________________________


### 3rd-order polynomials vision(각 parameter의 계수를 한번 추정하여 rid계산하는 경우)

In [10]:
fitting_paramters

intercept     7.102585
marriage      0.330761
family_num    0.058637
age^1/10^0   -0.013523
age^2/10^1    0.012117
age^3/10^2   -0.001289
dtype: float64

----------------------------------------
### shock calculation

In [11]:
jobtype_list = data_df["jobtype"].value_counts().index
jobtype_list

Index(['employ', 'unemploy'], dtype='object')

In [12]:
group_sigma_dict = {} 
group_rid_dict = {}
for l in jobtype_list:
    group_sigma_df = pd.DataFrame()
    group_pvalue_df = pd.DataFrame()
    group_rid_df = pd.DataFrame()

    start_list = [22, 24, 26]
    for i in range(data_df["0110"].max()):
        df_y_star = pd.DataFrame()

        temp_edu = data_df.groupby(["0110", "jobtype"]).get_group((i+1, l))
        temp_edu = get_preprocessing_data.get_agecut_data(temp_edu, start_list[i])

        fer_results = analysis_method.fe_regression_sol(temp_edu, False)
        fitting_result = analysis_method.fitting_polynomial(fer_results, 3, False)

        fitting_paramters = pd.concat([fer_results[0][:3], fitting_result[1]], axis=1).sum(axis=1)

        for j in data_num:
            temp_edu_split = data_dict[j].groupby(["0110", "jobtype"]).get_group((i+1, l))
            temp_edu_split = get_preprocessing_data.get_agecut_data(temp_edu_split, start_list[i])

            log_y = np.log(temp_edu_split["labor income"])

            x_mat = analysis_method.get_reg_x(temp_edu_split).iloc[:, :3]
            poly_x_mat = analysis_method.polynomial_matrix(temp_edu_split["0107"], 3) .iloc[:, 1:]
            poly_x_mat.index = x_mat.index

            temp_real_data = pd.concat([x_mat, poly_x_mat], axis=1)

            f_hat = temp_real_data @ fitting_paramters

            log_y_star = log_y - f_hat
            df_y_star = pd.concat([df_y_star, log_y_star], axis=1)

        df_y_star.columns = data_num

        temp_varrid_list = []
        temp_rid_list = []
        for k in range(len(data_num)-1):
            temp_rid = df_y_star.iloc[:, :-(k+1)].values - df_y_star.iloc[:, k+1:].values
            temp_rid_list.append(temp_rid)

            temp_varrid_list.append(np.nanvar(temp_rid))

        var_rid_series = pd.Series(temp_varrid_list)

        rid_series = (pd.DataFrame(temp_rid_list[0], columns = data_num[1:]) * -1).mean()
        group_rid_df = pd.concat([group_rid_df, rid_series], axis=1)

        var_rid_x = np.ones([len(var_rid_series), 2]) * 2
        var_rid_x[:, 0] = var_rid_series.index.values + 1
        var_rid_y = var_rid_series.values

        sol = sm.OLS(var_rid_y, var_rid_x)
        sol_result = sol.fit()

        sigma_df = pd.DataFrame(sol_result.params, index=["permanent_shock", "idiosyncratic_temporary_shock"], columns=["sigma"])
        pvalue_df = pd.DataFrame(np.round(sol_result.pvalues, 3), index=["permanent_shock(p-value)", "idiosyncratic_temporary_shock(p-value)"], columns=["p-value"])

        group_sigma_df = pd.concat([group_sigma_df, sigma_df], axis=1)
        group_pvalue_df = pd.concat([group_pvalue_df, pvalue_df], axis=1)

    group_rid_df.columns = ["high_school", "college", "university"]
    group_sigma_df.columns = ["high_school", "college", "university"]
    group_pvalue_df.columns = ["high_school", "college", "university"]
    group_sigma_df = pd.concat([group_sigma_df, group_pvalue_df]).loc[["idiosyncratic_temporary_shock", "permanent_shock", "idiosyncratic_temporary_shock(p-value)", "permanent_shock(p-value)"],:]
    group_sigma_dict[l] = group_sigma_df
    group_rid_dict[l] = group_rid_df

In [13]:
pd.DataFrame.from_dict({(i,j): group_sigma_dict[i][j]
                        for i in group_sigma_dict.keys()
                        for j in group_sigma_dict[i].columns}, orient="index")

idiosyncratic_temporary_shock  permanent_shock  \
employ   high_school                       0.118801         0.009552   
         college                           0.115425         0.009234   
         university                        0.110894         0.005373   
unemploy high_school                       0.246465         0.003320   
         college                           0.216422         0.009279   
         university                        0.219169         0.006102   

                      idiosyncratic_temporary_shock(p-value)  \
employ   high_school                                     0.0   
         college                                         0.0   
         university                                      0.0   
unemploy high_school                                     0.0   
         college                                         0.0   
         university                                      0.0   

                      permanent_shock(p-value)  
employ   high_school                     0.000  
         college                         0.000  
         university                      0.006  
unemploy high_school                     0.138  
         college                         0.006  
         university                      0.119

----------------------------------------
### Beta & Correlation

In [14]:
ksi = group_rid_dict
ksi = pd.DataFrame.from_dict({(i,j): ksi[i][j]
                        for i in ksi.keys()
                        for j in ksi[i].columns}, orient="columns")
ksi

employ                         unemploy                     
   high_school   college university high_school   college university
02    0.046185 -0.007723  -0.004135   -0.034931 -0.062489  -0.114921
03    0.051696  0.079658   0.038504    0.143040  0.045593   0.069446
04   -0.007253 -0.011416   0.123869   -0.006947 -0.041169   0.188259
05    0.183846  0.248241   0.098184    0.198312  0.461081   0.198026
06    0.074621  0.018812   0.024680    0.065145 -0.132948   0.091657
07    0.110557  0.084509   0.093107    0.033243  0.139282  -0.069752
08    0.061650  0.113645   0.085528   -0.007337  0.020929   0.075896
09    0.102706  0.082561   0.083688    0.113995  0.237726   0.033196
10    0.069169  0.156548   0.060283   -0.018754 -0.009588   0.140957
11    0.043107 -0.000118   0.060478    0.032049 -0.017047   0.035360
12   -0.037004  0.018395  -0.011746   -0.086511 -0.072865  -0.041915
13    0.078648 -0.010200   0.044826    0.088239  0.066164   0.039282
14    0.057905  0.018593   0.061400    0.074062  0.073846   0.026619
15    0.059695  0.029433   0.039632    0.054800 -0.009048  -0.044943
16    0.076983  0.068249   0.026502    0.017589 -0.011003   0.061532
17    0.072928  0.032285   0.015562    0.032443  0.030293   0.064318
18    0.034123  0.018711   0.009463    0.003508  0.111404   0.017210
19    0.045623  0.041510   0.029719    0.062356  0.010156  -0.014644
20    0.043208  0.054544   0.008340    0.007353  0.032235   0.084580
21    0.020653  0.007829   0.028216    0.061089 -0.023793   0.005920

In [15]:
kospi_return

,코스피
Symbol Name,
1,-42.21
2,49.47
3,82.78
4,-49.11
5,37.47
6,-9.54
7,29.19
8,10.51
9,55.99


In [16]:
kospi_return.index = data_num
kospi_ret = kospi_return.copy() / 100

beta_dict = {}
for j in group_rid_dict.keys():
    beta_list = []
    pvalue_list = []
    for i in group_rid_dict[j].columns:
        temp_eta = (kospi_ret - 0.02) - np.mean(kospi_ret - 0.02) 
        temp_beta_model = sm.OLS(ksi[j][i], temp_eta.loc[ksi[j].index])
        temp_beta_results = temp_beta_model.fit()
        #print("----------------------------------------------------------")
        #print(i, j)
        #print(temp_beta_results.summary())

        beta_list.append(temp_beta_results.params[0])
        pvalue_list.append(temp_beta_results.pvalues[0])

    beta = pd.DataFrame(beta_list, index=["high_school", "college", "university"], columns=["Beta"])
    pvalue = pd.DataFrame(pvalue_list, index=["high_school", "college", "university"], columns=["p-value"])
    
    beta_dict[j] = pd.concat([beta, pvalue], axis=1)

In [17]:
pd.DataFrame.from_dict({(i,j): beta_dict[i][j]
                        for i in beta_dict.keys()
                        for j in beta_dict[i].columns}, orient="index")

high_school   college  university
employ   Beta        0.089473  0.062426    0.022334
         p-value     0.090810  0.296318    0.605241
unemploy Beta        0.133833  0.179663   -0.024847
         p-value     0.009246  0.053918    0.705034

In [18]:
corr_dict = {}
for j in group_rid_dict.keys():
    corr_list = []
    for i in [0, 1, 2]:
        corr_list.append(np.corrcoef(temp_eta.loc[ksi[j].index, :].코스피, ksi[j][ksi[j].columns[i]])[0, 1])

    corr_dict[j] = pd.DataFrame(corr_list, index=ksi[j].columns, columns=["correlation coefficent"])

In [19]:
corr_df = pd.DataFrame.from_dict({(i,j): corr_dict[i][j]
                        for i in corr_dict.keys()
                        for j in corr_dict[i].columns}, orient="index")
corr_df.index = corr_df.index.levels[0]
corr_df

,high_school,college,university
employ,0.522984,0.239909,0.083220
unemploy,0.608529,0.422412,-0.148281


_______________________________


### 3rd-order polynomials vision(각 parameter의 계수를 매번 추정하여 rid계산하는 경우)

In [20]:
group_sigma_dict = {} 
group_rid_dict = {}
for l in jobtype_list:
    group_sigma_df = pd.DataFrame()
    group_pvalue_df = pd.DataFrame()
    group_rid_df = pd.DataFrame()

    start_list = [22, 24, 26]
    for i in range(data_df["0110"].max()):
        df_y_star = pd.DataFrame()

        for j in data_num:
            temp_edu_split = data_dict[j].groupby(["0110", "jobtype"]).get_group((i+1, l))
            temp_edu_split = get_preprocessing_data.get_agecut_data(temp_edu_split, start_list[i])

            fer_results = analysis_method.fe_regression_sol(temp_edu_split, False)
            fitting_result = analysis_method.fitting_polynomial(fer_results, 3, False)

            fitting_paramters = pd.concat([fer_results[0][:3], fitting_result[1]], axis=1).sum(axis=1)

            log_y = np.log(temp_edu_split["labor income"])

            x_mat = analysis_method.get_reg_x(temp_edu_split).iloc[:, :3]
            poly_x_mat = analysis_method.polynomial_matrix(temp_edu_split["0107"], 3) .iloc[:, 1:]
            poly_x_mat.index = x_mat.index

            temp_real_data = pd.concat([x_mat, poly_x_mat], axis=1)

            f_hat = temp_real_data @ fitting_paramters

            log_y_star = log_y - f_hat
            df_y_star = pd.concat([df_y_star, log_y_star], axis=1)

        df_y_star.columns = data_num

        temp_varrid_list = []
        temp_rid_list = []
        for k in range(len(data_num)-1):
            temp_rid = df_y_star.iloc[:, :-(k+1)].values - df_y_star.iloc[:, k+1:].values
            temp_rid_list.append(temp_rid)

            temp_varrid_list.append(np.nanvar(temp_rid))

        var_rid_series = pd.Series(temp_varrid_list)

        rid_series = (pd.DataFrame(temp_rid_list[0], columns = data_num[1:]) * -1).mean()
        group_rid_df = pd.concat([group_rid_df, rid_series], axis=1)

        var_rid_x = np.ones([len(var_rid_series), 2]) * 2
        var_rid_x[:, 0] = var_rid_series.index.values + 1
        var_rid_y = var_rid_series.values

        sol = sm.OLS(var_rid_y, var_rid_x)
        sol_result = sol.fit()

        sigma_df = pd.DataFrame(sol_result.params, index=["permanent_shock", "idiosyncratic_temporary_shock"], columns=["sigma"])
        pvalue_df = pd.DataFrame(np.round(sol_result.pvalues, 3), index=["permanent_shock(p-value)", "idiosyncratic_temporary_shock(p-value)"], columns=["p-value"])

        group_sigma_df = pd.concat([group_sigma_df, sigma_df], axis=1)
        group_pvalue_df = pd.concat([group_pvalue_df, pvalue_df], axis=1)

    group_rid_df.columns = ["high_school", "college", "university"]
    group_sigma_df.columns = ["high_school", "college", "university"]
    group_pvalue_df.columns = ["high_school", "college", "university"]
    group_sigma_df = pd.concat([group_sigma_df, group_pvalue_df]).loc[["idiosyncratic_temporary_shock", "permanent_shock", "idiosyncratic_temporary_shock(p-value)", "permanent_shock(p-value)"],:]
    group_sigma_dict[l] = group_sigma_df
    group_rid_dict[l] = group_rid_df

In [21]:
pd.DataFrame.from_dict({(i,j): group_sigma_dict[i][j]
                        for i in group_sigma_dict.keys()
                        for j in group_sigma_dict[i].columns}, orient="index")

idiosyncratic_temporary_shock  permanent_shock  \
employ   high_school                       0.124192         0.007717   
         college                           0.119528         0.006086   
         university                        0.109196         0.004746   
unemploy high_school                       0.244837         0.002113   
         college                           0.201907         0.009936   
         university                        0.230042         0.001670   

                      idiosyncratic_temporary_shock(p-value)  \
employ   high_school                                     0.0   
         college                                         0.0   
         university                                      0.0   
unemploy high_school                                     0.0   
         college                                         0.0   
         university                                      0.0   

                      permanent_shock(p-value)  
employ   high_school                     0.000  
         college                         0.006  
         university                      0.007  
unemploy high_school                     0.337  
         college                         0.003  
         university                      0.678

----------------------------------------
### Beta & Correlation

In [22]:
ksi = group_rid_dict
ksi = pd.DataFrame.from_dict({(i,j): ksi[i][j]
                        for i in ksi.keys()
                        for j in ksi[i].columns}, orient="columns")
ksi

employ                         unemploy                     
   high_school   college university high_school   college university
02    0.077307  0.007060   0.022730    0.040909  0.156076   0.005130
03   -0.018387  0.061780   0.047553    0.005384 -0.049573  -0.041481
04    0.056486 -0.009604   0.005576    0.018418  0.034422   0.114857
05   -0.003225  0.058041   0.050719   -0.013875 -0.006322   0.032741
06    0.097782  0.093601   0.051561    0.029606 -0.042434   0.053710
07    0.063880  0.028833   0.046059    0.035545  0.065694   0.020012
08    0.034809  0.131743   0.029715   -0.013500 -0.007963   0.020394
09    0.076296  0.055375   0.057052    0.021184  0.196528  -0.061461
10    0.017112  0.078261   0.071838    0.027959 -0.031685   0.094217
11    0.044771 -0.042233   0.030411   -0.020225 -0.073494  -0.024128
12    0.028498  0.081025   0.017099    0.029105  0.015725   0.021598
13    0.044679  0.015854   0.028132    0.025836  0.058367   0.045075
14    0.048623 -0.005526   0.037544    0.005113  0.088588   0.025133
15    0.033429  0.005953   0.029772    0.018511 -0.057241  -0.003764
16    0.056787  0.014192   0.032561   -0.017382  0.041177   0.054003
17    0.051454  0.019162   0.026627   -0.000635  0.011191   0.037408
18   -0.004460  0.033009  -0.000410    0.036581  0.024343   0.006999
19    0.008339  0.053563   0.010122    0.009631 -0.000807   0.007330
20    0.010928 -0.038057   0.017194    0.017068 -0.033707   0.049569
21    0.015934  0.010387  -0.004847    0.010856 -0.003996  -0.018730

In [23]:
ksi = group_rid_dict

kospi_return.index = data_num
kospi_ret = kospi_return.copy() / 100


beta_dict = {}
for j in group_rid_dict.keys():
    beta_list = []
    pvalue_list = []
    for i in group_rid_dict[j].columns:
        temp_eta = (kospi_ret - 0.02) - np.mean(kospi_ret - 0.02) 
        temp_beta_model = sm.OLS(group_rid_dict[j][i], temp_eta.loc[group_rid_dict[j].index])
        temp_beta_results = temp_beta_model.fit()
        #print("----------------------------------------------------------")
        #print(i, j)
        #print(temp_beta_results.summary())

        beta_list.append(temp_beta_results.params[0])
        pvalue_list.append(temp_beta_results.pvalues[0])

    beta = pd.DataFrame(beta_list, index=["high_school", "college", "university"], columns=["Beta"])
    pvalue = pd.DataFrame(pvalue_list, index=["high_school", "college", "university"], columns=["p-value"])
    
    beta_dict[j] = pd.concat([beta, pvalue], axis=1)

In [24]:
pd.DataFrame.from_dict({(i,j): beta_dict[i][j]
                        for i in beta_dict.keys()
                        for j in beta_dict[i].columns}, orient="index")

high_school   college  university
employ   Beta        0.001458  0.006104    0.033560
         p-value     0.967156  0.879374    0.201583
unemploy Beta       -0.004686  0.065023   -0.072937
         p-value     0.780821  0.201966    0.024629

In [25]:
corr_dict = {}
for j in group_rid_dict.keys():
    corr_list = []
    for i in [0, 1, 2]:
        corr_list.append(np.corrcoef(temp_eta.loc[ksi[j].index, :].코스피, ksi[j][ksi[j].columns[i]])[0, 1])

    corr_dict[j] = pd.DataFrame(corr_list, index=ksi[j].columns, columns=["correlation coefficent"])

In [26]:
corr_df = pd.DataFrame.from_dict({(i,j): corr_dict[i][j]
                        for i in corr_dict.keys()
                        for j in corr_dict[i].columns}, orient="index")
corr_df.index = corr_df.index.levels[0]
corr_df

,high_school,college,university
employ,-0.093726,-0.021955,0.404334
unemploy,-0.143891,0.277935,-0.602745
